In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish


In [2]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

sys.path.append("../../")

from src.ad_hmm import sku_predict
from src.ad_stl_model import stl_model
from src.ad_stl_prophet import stl_model
from src.utils import *

Importing plotly failed. Interactive plots will not work.


In [3]:
%load_ext autoreload 
%autoreload 2

### Input data

In [4]:
train_test = pd.read_csv("../../data/train_validation_marker.csv")
train_test["SALES_DATE"] = pd.to_datetime(train_test["SALES_DATE"])
print (train_test.shape)

train_test["if_promo"] = (~train_test["PROMO_PRICE"].isin(["?"])).astype("int")
train_test["if_competitor"] = (~train_test["COMPETITOR_PRICE"].isin(["?"])).astype("int")

train_test["min_sales_price"] = pd.to_numeric(np.where(train_test.if_promo>0, train_test.PROMO_PRICE,\
                                                       train_test.RETAIL_PRICE))

train_test["comp_price_clean"] = pd.to_numeric(np.where(train_test.if_competitor>0,train_test.COMPETITOR_PRICE, np.nan))
train_test["price_delta"] = np.where(train_test.if_competitor>0, train_test.min_sales_price - train_test.comp_price_clean,\
                                   -train_test.min_sales_price)

(846632, 15)


### Pivot data

In [5]:
train_test_pivot = train_test.pivot(index='SALES_DATE', columns='Encoded_SKU_ID',\
                   values=['DAILY_UNITS',"if_promo","if_competitor","min_sales_price","price_delta",\
                          "validation","validation_clean","RETAIL_PRICE"])
train_test_pivot.columns = train_test_pivot.columns.swaplevel(0, 1)

### Test code for 1 sku

In [6]:
sku_id = 50
train_test_sku = train_test_pivot.loc[:,(sku_id,)].copy()
train_test_sku["ad_time"] = list(range(1,train_test_sku.shape[0]+1))

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/4124826544.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  train_test_sku = train_test_pivot.loc[:,(sku_id,)].copy()


In [7]:
exog_cols = ["if_promo","if_competitor","min_sales_price","price_delta","RETAIL_PRICE"]
stl_model1 = stl_model(sku_id = sku_id,\
                      train_test_sku = train_test_sku,\
                      n_lags_y = 30, \
                      n_lags_exog = 14, \
                      pred_cols = exog_cols)

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
m_dict, y_dict = stl_model1.fit_models();

22:58:35 - cmdstanpy - INFO - Chain [1] start processing
22:58:35 - cmdstanpy - INFO - Chain [1] done processing


(50, 'none', 'prophet'), Train RMSE = 13.5

(50, 'STL_resid', 'auto_reg'), Train RMSE = 9.189
(50, 'STL_resid', 'xgb_340'), Train RMSE = 11.06
(50, 'STL_resid', 'xgb_1000'), Train RMSE = 10.37

(50, 'STL_trend', 'auto_reg'), Train RMSE = 0.0005141
(50, 'STL_trend', 'xgb_340'), Train RMSE = 1.13
(50, 'STL_trend', 'xgb_1000'), Train RMSE = 1.13

(50, 'STL_seasonal', 'auto_reg'), Train RMSE = 9.646
(50, 'STL_seasonal', 'xgb_340'), Train RMSE = 7.19
(50, 'STL_seasonal', 'xgb_1000'), Train RMSE = 7.8

(50, 'MSTL_resid', 'auto_reg'), Train RMSE = 6.781
(50, 'MSTL_resid', 'xgb_340'), Train RMSE = 12.55
(50, 'MSTL_resid', 'xgb_1000'), Train RMSE = 12.4

(50, 'MSTL_trend', 'auto_reg'), Train RMSE = 0.0004901
(50, 'MSTL_trend', 'xgb_340'), Train RMSE = 3.413
(50, 'MSTL_trend', 'xgb_1000'), Train RMSE = 3.413

(50, 'MSTL_seasonal_7', 'auto_reg'), Train RMSE = 0.4017
(50, 'MSTL_seasonal_7', 'xgb_340'), Train RMSE = 6.306
(50, 'MSTL_seasonal_7', 'xgb_1000'), Train RMSE = 6.023

(50, 'MSTL_seasonal_

In [18]:
min_models, y_pred = stl_model1.get_result();

In [19]:
min_models

,component,model_name,model,train_rmse,rnk
0,none,prophet,<prophet.forecaster.Prophet object at 0x168722...,13.501301,1.0
1,STL_resid,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,9.189268,1.0
4,STL_trend,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.000514,1.0
8,STL_seasonal,xgb_340,ForecastingCascade(estimator=XGBRegressor(base...,7.189561,1.0
10,MSTL_resid,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,6.780919,1.0
13,MSTL_trend,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.00049,1.0
16,MSTL_seasonal_7,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.401686,1.0
19,MSTL_seasonal_30,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,1.029333,1.0
23,MSTL_seasonal_365,xgb_340,ForecastingCascade(estimator=XGBRegressor(base...,7.176495,1.0


In [20]:
print (np.mean((y_pred["actual"]-y_pred["STL_prediction"])**2)**0.5,
np.mean((y_pred["actual"]-y_pred["MSTL_prediction"])**2)**0.5,
      np.mean((y_pred["actual"]-y_pred["prophet_prediction"])**2)**0.5)

12.756134433902387 22.941797893647582 10.56564263929332


### Run for all SKUs

In [21]:
from joblib import Parallel, delayed
from IPython.utils import io
from tqdm import tqdm

In [22]:
def fit_one_sku(train_test_pivot1,sku_id1, n_lags_y1, n_lags_exog1, exog_cols1):
    train_test_sku1 = train_test_pivot1.loc[:,(sku_id1,)].copy()
    train_test_sku1["ad_time"] = list(range(1,train_test_sku1.shape[0]+1))
    stl_model1 = stl_model(sku_id = sku_id1,\
                      train_test_sku = train_test_sku1,\
                      n_lags_y = n_lags_y1, \
                      n_lags_exog = n_lags_exog1, \
                      pred_cols = exog_cols1)
    try:
        with io.capture_output() as captured:
            m_dict, y_dict = stl_model1.fit_models();
            min_models, y_pred = stl_model1.get_result();
        print (f"Successful for {sku_id1}")
        return (min_models, y_pred)
    except:
        print (f"Error for {sku_id1}")

In [23]:
clean_SKUs = train_test[train_test.validation_clean==True]["Encoded_SKU_ID"].unique()

In [38]:
all_preds = Parallel(n_jobs=4)(delayed(fit_one_sku)(train_test_pivot1 = train_test_pivot,\
                                                    sku_id1 = sku_id_i,\
                                                    n_lags_y1 = 30,\
                                                    n_lags_exog1 = 14,\
                                                    exog_cols1 = exog_cols) \
                               for sku_id_i in tqdm(clean_SKUs))

  1%|█                                                                                                                                         | 4/545 [00:01<02:18,  3.90it/s]Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
Importing plotly failed. Interactive plots will not work.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
Importing plotly failed. Interactive plots will not work.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth m

  8%|██████████▋                                                                                                                          | 44/545 [58:00<14:50:08, 106.60s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  9%|███████████▋                                                                                                                        | 48/545 [1:02:13<12:54:05, 93.45s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1k

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
 16%|█████████████████████▍                                                                                                               | 88/545 [1:40:58<7:40:45, 60.49s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipyker

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
 23%|███████████████████████████████                                                                                                     | 128/545 [2:20:33<6:59:14, 60.32s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
 24%|███████████████████████████████▉                  

 31%|████████████████████████████████████████▋                                                                                           | 168/545 [3:04:01<6:45:48, 64.59s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
 32%|█████████████████████████████████████████▋                                                                                          | 172/545 [3:07:53<6:28:58, 62.57s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1k

...

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
 69%|███████████████████████████████████████████████████████████████████████████████████████████                                         | 376/545 [6:33:14<2:12:03, 46.89s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
 70%|██████████████████████████████████████████████████

 76%|████████████████████████████████████████████████████████████████████████████████████████████████████▊                               | 416/545 [7:08:47<1:27:33, 40.73s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
 77%|█████████████████████████████████████████████████████████████████████████████████████████████████████▋                              | 420/545 [7:11:59<1:29:18, 42.86s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1k

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
 84%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████                     | 460/545 [7:42:04<57:41, 40.73s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipyker

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
 92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏          | 500/545 [9:47:42<3:03:15, 244.35s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
 92%|██████████████████████████████████████████████████

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 540/545 [10:14:25<03:48, 45.63s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_36812/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 545/545 [10:15:23<00:00, 67.75s/it]
/var/folders/kb/v0p0ypbd0wx1q2qb1

In [39]:
len(all_preds)

545

In [37]:
# check = pd.Series(y_pred_df.groupby("Encoded_SKU_ID").apply(lambda x: (np.mean((x["STL_prediction"]-x["actual"])**2)**0.5,\
#                                          np.mean((x["MSTL_prediction"]-x["actual"])**2)**0.5,
#                                                     np.mean((x["prophet_prediction"]-x["actual"])**2)**0.5)\
#                                          ))
# check.to_list()

[(1.2843884237969407, 1.1327497861715485, 1.1286186895140702),
 (2.400849032058559, 2.6401230498190036, 3.871131689349082),
 (1.209259026232583, 1.9956175952808217, 1.432526021492592),
 (1.2388502748636492, 1.3069716617419507, 1.5352251743749514)]

In [40]:
import pickle
with open('../../data/stl_prophet_results_v1.pkl', 'wb') as f:
    pickle.dump(all_preds, f)

In [41]:
y_pred_df = pd.DataFrame()
for i in range(len(all_preds)):
    if all_preds[i]!= None:
        y_pred_df = pd.concat([y_pred_df,all_preds[i][1]])

In [42]:
y_pred_df.head()

,STL_prediction,MSTL_prediction,prophet_prediction,Encoded_SKU_ID,actual
SALES_DATE,,,,,
2022-07-25,1.139742,1.562445,1.662813,1,0
2022-07-26,2.706996,1.328076,1.256640,1,0
2022-07-27,1.287372,1.550463,1.314178,1,0
2022-07-28,1.401008,2.986880,1.533027,1,2
2022-07-29,2.200458,2.243526,2.250344,1,2


### RMSE

y_pred_df["Encoded_SKU_ID"].nunique()

validation = train_test[(train_test.validation==True) & (train_test.validation_clean==True)]
y_act = validation[["Encoded_SKU_ID","SALES_DATE","DAILY_UNITS"]].rename(columns = {"DAILY_UNITS":"actual"})
y_act.head()

y_pred_null = pd.read_csv("../../data/null_prediction_validation_result_v0.csv").drop(columns = "Unnamed: 0")
y_pred_null["SALES_DATE"] = pd.to_datetime(y_pred_null["SALES_DATE"])

y_pred_stl = y_pred_df.reset_index()[["SALES_DATE","STL_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"STL_prediction":"predicted"})

y_pred_mstl = y_pred_df.reset_index()[["SALES_DATE","MSTL_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"MSTL_prediction":"predicted"})

# y_act = y_pred_df.reset_index()[["SALES_DATE","actual","Encoded_SKU_ID"\
#                         ]]
y_act["actual"] = pd.to_numeric(y_act["actual"])

y_pred_null_subset = y_pred_null.loc[y_pred_null.Encoded_SKU_ID.isin(y_pred_stl.Encoded_SKU_ID.unique())]
len(y_pred_null),len(y_pred_null_subset)

rmse(y_act,y_pred_null),rmse(y_act,y_pred_null_subset)

rmse(y_act,y_pred_stl)

rmse(y_act,y_pred_mstl)

rmse_sku_stl = rmse_sku(y_act,y_pred_stl)
rmse_sku_mstl = rmse_sku(y_act,y_pred_mstl)
rmse_sku_null = rmse_sku(y_act,y_pred_null)

VERSION = "v2"
hmm_output = pd.read_csv(f"../../data/hmm_result_{VERSION}.csv",index_col=0)

y_act.dtypes

rmse_sku_hmm_dates = pd.DataFrame()
for i,col in enumerate(hmm_output.columns):
    if "pred" in col:
        y_pred_hmm = hmm_output.reset_index(names = "SALES_DATE")[["Encoded_SKU_ID","SALES_DATE",col\
                                                                  ]].rename(columns={col:"predicted"})
        y_pred_hmm["SALES_DATE"] = pd.to_datetime(y_pred_hmm["SALES_DATE"])
        rmse_sku_hmm_i = rmse_sku(y_act,y_pred_hmm)
        if i ==0:
            rmse_sku_hmm_dates = rmse_sku_hmm_i
        else:
            rmse_sku_hmm_dates = rmse_sku_hmm_dates.merge(rmse_sku_hmm_i, left_index = True, right_index = True,\
                                suffixes = ("",f"_{col.split('_')[-1]}"))

other_rmse = rmse_sku_stl.merge(rmse_sku_mstl,on=["Encoded_SKU_ID"],how="outer", suffixes = ("_stl","_mstl")\
                  ).merge(rmse_sku_null,on=["Encoded_SKU_ID"],how="outer", suffixes = ("","_null"))

y_act.Encoded_SKU_ID.nunique()

rmse_sku_join = rmse_sku_hmm_dates.merge(other_rmse,left_index=True,right_on=["Encoded_SKU_ID"],how="outer",\
                                        suffixes = ("_15","_null"),)

### Pick Best

rmse_cols = [col for col in rmse_sku_join.columns if "rmse_du" in col]
rmse_cols

rmse_sku_join["min_rmse"] = rmse_sku_join[rmse_cols].min(axis=1,skipna=True)

rmse_sku_join["argmin_period"] = np.argmin(rmse_sku_join[rmse_cols].values,axis=1)

rmse_sku_join["min_period"] = rmse_sku_join["argmin_period"].apply(lambda x: rmse_cols[x])

rmse_sku_join["min_rmse"].isna().sum()

hmm_output1 = hmm_output.reset_index(names="SALES_DATE")
hmm_output1["SALES_DATE"] = pd.to_datetime(hmm_output1["SALES_DATE"])

all_outputs = hmm_output1.merge(y_pred_df.reset_index(),on=["Encoded_SKU_ID","SALES_DATE"],how="outer"\
                                            ).merge(y_pred_null,on=["Encoded_SKU_ID","SALES_DATE"],how="outer",\
                                                   suffixes = ("_15","_null"))

all_outputs

period_col_map= {
    "rmse_du_15":"predicted_15", "rmse_du_30":"predicted_30", "rmse_du_45":"predicted_45",\
    "rmse_du_60":"predicted_60", "rmse_du_90":"predicted_90", "rmse_du_max":"predicted_max",\
    "rmse_du_stl":"STL_prediction","rmse_du_mstl":"MSTL_prediction","rmse_du_null":"predicted_null"
}

y_pred_best = pd.DataFrame()
cnt = 0
for sku_id,best_period in zip(rmse_sku_join.index,rmse_sku_join["min_period"]):
    best_col = period_col_map[best_period]
    mask_sku = all_outputs.Encoded_SKU_ID==sku_id
    y_pred_best_sku = all_outputs.loc[mask_sku][["Encoded_SKU_ID","SALES_DATE",best_col\
                                                                  ]].rename(columns={best_col:"predicted"})
    y_pred_best = pd.concat([y_pred_best,y_pred_best_sku])

y_act.Encoded_SKU_ID.nunique(),y_pred_null.Encoded_SKU_ID.nunique(),rmse_sku_join.index.nunique()

a,b = rmse(y_act,y_pred_null),rmse(y_act,y_pred_best)
a,b

(a-b)/a

rmse_sku_join["min_period"].value_counts()